# Dump salaries from schedule J in Portland, OR (continuing from part 1)

In [102]:
import unicodecsv as csv
from irsx.xmlrunner import XMLRunner
import pandas as pd

In [103]:
df = pd.read_csv('flefilers1.csv', delimiter = ',', dtype='unicode')


In [104]:
df.head()


,Unnamed: 0,RETURN_ID,FILING_TYPE,EIN,TAX_PERIOD_x,SUB_DATE,TAXPAYER_NAME,RETURN_TYPE,DLN,OBJECT_ID,NAME,ICO,STREET,CITY,STATE,ZIP,INCOME_AMT,ASSET_AMT,TAX_PERIOD_y
0,8263,16318403,EFILE,461856500,201812,5/20/19 19:54,FIGHT MINISTRIES INC,990,93493042007079,201920429349300707,FIGHT MINISTRIES INC,NaN,9920 MOORE RD,LAKELAND,FL,33809-0000,228920,178074,201812
1,8362,16365643,EFILE,462321723,201812,5/31/19 12:30,OPEN ARMS VILLAGE INC,990,93493134090509,201901349349309050,OPEN ARMS VILLAGE INC,% KARLA GRIMSLEY,1839 NE 8TH RD,OCALA,FL,34470-4234,230632,156069,201812
2,8421,16395542,EFILE,462609888,201812,6/7/19 23:15,TOWN OF CELEBRATION MARATHON AND HALF MARATHON,990,93493136004419,201911369349300441,ROAD RUNNERS CLUB OF AMERICA,% KEN ZAMBITO - PRESIDENT,52 RILEY ROAD 428,CELEBRATION,FL,34747-5420,225198,159439,201812
3,8796,16465836,EFILE,464001708,201812,7/8/19 6:00,ST MARTIN DE PORRES OUTREACH COMMUNITY MINISTR...,990,93493133009109,201901339349300910,ST MARTIN DE PORRES OUTREACH COMMUNITY MINISTR...,% BARBARA DURKIN,PO BOX 50754,FORT MYERS,FL,33994-0754,201717,102463,201812
4,8822,16436546,EFILE,464117051,201812,6/24/19 16:39,LUTZ BASEBALL INC,990EZ,93492149003409,201901499349200340,LUTZ BASEBALL,% WILLIAM PATTERSON,18824 CHOPIN DR,LUTZ,FL,33558-2876,196359,89636,201812


In [105]:
# read the whole file of orgs with efilings from part 1 here, it's not very long
file_rows = [] 
# We're using the output of part 1
with open('flefilers1.csv', 'rb') as infile:
    reader = csv.DictReader(infile)
    for row in reader:
        file_rows.append(row)
        

In [106]:
# the name of the output file
outfilename ="employees_simple.csv"
outfile = open(outfilename , 'wb')

# the header rows as they'll appear in the output
headers = ["period", "ein", "object_id", "taxpayer_name", "name", "business_name1", "business_name2", "title", "base_pay", "bonus_pay", "other_pay", "deferred_pay", "nontax_pay", "added_together"]
# start up a dictwriter, ignore extra rows
dw = csv.DictWriter(outfile, headers, extrasaction='ignore')
dw.writeheader()

In [107]:
# get an XMLRunner -- this is what actually does the parsing
xml_runner = XMLRunner()

## Figure out what to extract

Data from each repeating group should go to it's own file, otherwise it won't make sense.

To figure out what to capture, I started by looking at schedule J: http://www.irsx.info/#IRS990ScheduleJ
Then I went to the table details and picked the rows I wanted from the repeating group:
http://www.irsx.info/metadata/groups/SkdJRltdOrgOffcrTrstKyEmpl.html

Note that it's common for director/employee names in schedule J to get listed as businessname.

Also note that IRSx checks to see if a file has been downloaded before fetching it. Running this the first time will be slow if the filings aren't already downloaded, but much faster if they've already been downloaded.


In [108]:
DEMO_MAX = 10
num_rows = 0

for row in file_rows:
    num_rows += 1
    this_object_id = row['OBJECT_ID']
    parsed_filing = xml_runner.run_filing(this_object_id)
    
    # if it somehow busted, just note it and continue
    if not parsed_filing:
        print("Skipping filing %s(filings with pre-2013 schemas are skipped)\n row details: %s" % (this_object_id, row))
        continue 
    
    # Is there a Schedule J there at all?
    schedule_list = parsed_filing.list_schedules()
    if 'IRS990ScheduleJ' in schedule_list:
        
        # store the output in this dict
        outputdata = {}
        # assign some initial values from the input csv
        outputdata['period'] = row['TAX_PERIOD_x']
        outputdata['ein'] = row['EIN']
        outputdata['object_id'] = row['OBJECT_ID']
        outputdata['taxpayer_name'] = row['TAXPAYER_NAME']
        
        # some schedules can appear multiple times, but schedule j only appears once
        # so we grab the first one 
        parsed_skedj = parsed_filing.get_parsed_sked('IRS990ScheduleJ')[0] 
        # now use the table name we looked up -- that's where we find it
        try:
            # repeating groups are returned as an array of dicts
            employee_groups = parsed_skedj['groups']['SkdJRltdOrgOffcrTrstKyEmpl']
        except KeyError:
            print("No SkdJRltdOrgOffcrTrstKyEmpl found in %s skipping" % this_object_id)
            continue
          
        # read through each employee and pull out the data we want
        for employee_group in employee_groups:
            
            # That leaves the following values to come from schedule J if there is one
            # "name", "business_name1", "business_name2", "title", "org_comp", "related_comp"
            # those keys come from the headers we gave dictwriter before
            outputdata['name'] = employee_group.get('PrsnNm')
            outputdata['business_name1'] = employee_group.get('BsnssNmLn1Txt')
            outputdata['business_name2'] = employee_group.get('BsnssNmLn2Txt')
            outputdata['title'] = employee_group.get('TtlTxt')
            outputdata['base_pay'] = employee_group.get('BsCmpnstnFlngOrgAmt')
            outputdata['bonus_pay'] = employee_group.get('BnsFlngOrgnztnAmnt')
            outputdata['other_pay'] = employee_group.get('OthrCmpnstnFlngOrgAmt')
            outputdata['deferred_pay'] = employee_group.get('DfrrdCmpnstnFlngOrgAmt')
            outputdata['nontax_pay'] = employee_group.get('NntxblBnftsFlngOrgAmt')
            outputdata['added_together'] = employee_group.get('TtlCmpnstnFlngOrgAmt')
#             outputdata['relatedorg_base_pay'] = employee_group.get('CmpnstnBsdOnRltdOrgsAmt')
#             outputdata['relatedorg_bonus_pay'] = employee_group.get('BnsRltdOrgnztnsAmt')
#             outputdata['relatedorg_other_pay'] = employee_group.get('OthrCmpnstnRltdOrgsAmt')
#             outputdata['relatedorg_deferred_pay'] = employee_group.get('DfrrdCmpRltdOrgsAmt')
#             outputdata['relatedorg_nontax_pay'] = employee_group.get('NntxblBnftsRltdOrgsAmt')
#             outputdata['relatedorg_added_together'] = employee_group.get('TtlCmpnstnRltdOrgsAmt')
            dw.writerow(outputdata)
        
    else:
        print("No schedule J in filing %s, skipping" % this_object_id)

    # Don't run endlessly during a demo:
#     if(num_rows > DEMO_MAX):
#         break
#     if num_rows%100==0:
#         print("Processed %s filings" % num_rows)

No schedule J in filing 201920429349300707, skipping
No schedule J in filing 201901349349309050, skipping
No schedule J in filing 201911369349300441, skipping
No schedule J in filing 201901339349300910, skipping
No schedule J in filing 201901499349200340, skipping
No schedule J in filing 201940449349100009, skipping
No schedule J in filing 201920929349200237, skipping
No schedule J in filing 201910539349300211, skipping
No schedule J in filing 201910919349200531, skipping
No schedule J in filing 201900669349100605, skipping
No schedule J in filing 201941399349300254, skipping
No schedule J in filing 201940999349100119, skipping
No schedule J in filing 201911439349300421, skipping
No schedule J in filing 201941309349300309, skipping
No schedule J in filing 201901209349301605, skipping
No schedule J in filing 201900289349100355, skipping
No schedule J in filing 201911409349100256, skipping
No schedule J in filing 201900929349300335, skipping
No schedule J in filing 201931349349308523, sk

No schedule J in filing 201910469349200546, skipping
No schedule J in filing 201931309349302763, skipping
No schedule J in filing 201930689349301103, skipping
Filing version 2018v3.0 isn't supported for this operation
No schedule J in filing 201941239349101104, skipping
No schedule J in filing 201911269349301721, skipping
No schedule J in filing 201900749349300710, skipping
No schedule J in filing 201911289349201181, skipping
No schedule J in filing 201931359349301823, skipping
No schedule J in filing 201911139349200201, skipping
No schedule J in filing 201901349349300240, skipping
No schedule J in filing 201900999349300620, skipping
No schedule J in filing 201941359349203874, skipping
No schedule J in filing 201901349349203520, skipping
No schedule J in filing 201920949349100402, skipping
No schedule J in filing 201941289349200119, skipping
No schedule J in filing 201911359349200711, skipping
No schedule J in filing 201941289349200914, skipping
No schedule J in filing 2019204393492005

No schedule J in filing 201901299349200920, skipping
No schedule J in filing 201941309349202594, skipping
No schedule J in filing 201911139349300111, skipping
No schedule J in filing 201941449349100204, skipping
No schedule J in filing 201900599349300615, skipping
No schedule J in filing 201910659349200721, skipping
No schedule J in filing 201900749349300010, skipping
Filing version 2018v3.0 isn't supported for this operation
No schedule J in filing 201911239349101131, skipping
No schedule J in filing 201900929349100500, skipping
No schedule J in filing 201911229349101231, skipping
No schedule J in filing 201931139349100913, skipping
Filing version 2018v3.0 isn't supported for this operation
No schedule J in filing 201931239349100808, skipping
No schedule J in filing 201931199349301358, skipping
No schedule J in filing 201930889349300318, skipping
No schedule J in filing 201900749349300215, skipping
Filing version 2018v3.0 isn't supported for this operation
No schedule J in filing 2019

No schedule J in filing 201910439349300951, skipping
Filing version 2018v3.0 isn't supported for this operation
No schedule J in filing 201921129349301132, skipping
No schedule J in filing 201930199349200703, skipping
No schedule J in filing 201921079349200502, skipping
No schedule J in filing 201920719349100222, skipping
No schedule J in filing 201921029349100222, skipping
No schedule J in filing 201910389349200631, skipping
No schedule J in filing 201920919349200602, skipping
Filing version 2018v3.0 isn't supported for this operation
No schedule J in filing 201941389349200204, skipping
No schedule J in filing 201920819349200657, skipping
No schedule J in filing 201940889349300534, skipping
No schedule J in filing 201901339349202395, skipping
No schedule J in filing 201901339349200790, skipping
Filing version 2018v3.0 isn't supported for this operation
No schedule J in filing 201921129349200937, skipping
No schedule J in filing 201941299349302364, skipping
No schedule J in filing 2019

No schedule J in filing 201921239349200442, skipping
No schedule J in filing 201920759349300012, skipping
No schedule J in filing 201931169349300613, skipping
No schedule J in filing 201921349349201617, skipping
No schedule J in filing 201900299349100510, skipping
No schedule J in filing 201910719349200221, skipping
No schedule J in filing 201941309349201769, skipping
No schedule J in filing 201931269349200218, skipping
No schedule J in filing 201930989349300648, skipping
No schedule J in filing 201940619349200204, skipping
No schedule J in filing 201930589349200708, skipping
No schedule J in filing 201940889349300849, skipping
No schedule J in filing 201920869349200332, skipping
No schedule J in filing 201901349349202790, skipping
No schedule J in filing 201900849349200420, skipping
No schedule J in filing 201921269349200502, skipping
No schedule J in filing 201900679349200410, skipping
No schedule J in filing 201931359349303123, skipping
No schedule J in filing 201920459349200912, sk

...

No schedule J in filing 201713189349101671, skipping
No schedule J in filing 201711889349200511, skipping
No schedule J in filing 201730139349200823, skipping
No schedule J in filing 201720969349200602, skipping
No schedule J in filing 201712609349200101, skipping
No schedule J in filing 201701649349100905, skipping
No schedule J in filing 201720899349200622, skipping
No schedule J in filing 201643429349200019, skipping
No schedule J in filing 201741669349301344, skipping
No schedule J in filing 201701789349200000, skipping
No schedule J in filing 201702219349100515, skipping
No schedule J in filing 201732239349100503, skipping
No schedule J in filing 201732199349300768, skipping
No schedule J in filing 201722149349300522, skipping
No schedule J in filing 201710399349300331, skipping
No schedule J in filing 201700909349300855, skipping
No schedule J in filing 201700669349300545, skipping
No schedule J in filing 201710279349300736, skipping
No schedule J in filing 201722239349301527, sk

No schedule J in filing 201700459349302015, skipping
No schedule J in filing 201731319349302613, skipping
No schedule J in filing 201701369349300320, skipping
No schedule J in filing 201700409349301435, skipping
No schedule J in filing 201731319349302563, skipping
No schedule J in filing 201623209349310317, skipping
No SkdJRltdOrgOffcrTrstKyEmpl found in 201731359349304173 skipping
No schedule J in filing 201603089349300140, skipping
No schedule J in filing 201731359349308103, skipping
No schedule J in filing 201741359349306224, skipping
No schedule J in filing 201730889349300508, skipping
No schedule J in filing 201603429349300725, skipping
No schedule J in filing 201720839349100502, skipping
No schedule J in filing 201740239349100004, skipping
No schedule J in filing 201740419349301079, skipping
No schedule J in filing 201740329349301009, skipping
No schedule J in filing 201603209349302190, skipping
No schedule J in filing 201720449349300627, skipping
No schedule J in filing 20170128

No schedule J in filing 201700449349300220, skipping
No schedule J in filing 201613589349300701, skipping
No schedule J in filing 201700239349300140, skipping
No schedule J in filing 201623379349300322, skipping
No schedule J in filing 201701089349201065, skipping
No schedule J in filing 201730139349300808, skipping
No schedule J in filing 201720169349200152, skipping
No schedule J in filing 201642889349200024, skipping
No schedule J in filing 201603079349200870, skipping
No schedule J in filing 201730139349200343, skipping
No schedule J in filing 201730259349100608, skipping
No schedule J in filing 201603199349203235, skipping
No schedule J in filing 201742089349300734, skipping
No schedule J in filing 201612579349200641, skipping
No schedule J in filing 201711819349200121, skipping
No schedule J in filing 201730949349300513, skipping
No schedule J in filing 201632589349300623, skipping
No schedule J in filing 201613339349300006, skipping
No schedule J in filing 201642329349300644, sk

No schedule J in filing 201633279349200608, skipping
No schedule J in filing 201623209349200412, skipping
No schedule J in filing 201633199349203083, skipping
No schedule J in filing 201612599349100636, skipping
No schedule J in filing 201643089349200719, skipping
No schedule J in filing 201622589349100422, skipping
No schedule J in filing 201642529349301009, skipping
No schedule J in filing 201622909349100002, skipping
No schedule J in filing 201613169349301641, skipping
No schedule J in filing 201642439349200704, skipping
No schedule J in filing 201802559349200230, skipping
No schedule J in filing 201623209349204482, skipping
No schedule J in filing 201643199349200449, skipping
No schedule J in filing 201632849349200023, skipping
No schedule J in filing 201613199349204696, skipping
No schedule J in filing 201622799349100002, skipping
No schedule J in filing 201643209349206539, skipping
No schedule J in filing 201613169349100846, skipping
No schedule J in filing 201623199349306502, sk

No schedule J in filing 201613019349200011, skipping
No schedule J in filing 201642849349200934, skipping
No schedule J in filing 201732709349200123, skipping
No schedule J in filing 201700629349200010, skipping
No schedule J in filing 201612389349200721, skipping


In [109]:
outfile.close()
fl_efilers = pd.read_csv(outfilename)

fl_efilers.to_csv(outfilename)


In [110]:
fl_efilers.head()